In [3]:
import numpy as np
import random

# 初始化参数
def initialize_parameters(numIn, numHide, numOut):
    # w1和b1为输入层与隐藏层之间的权重和偏置，w2和b2为隐藏层与输出层之间的权重和偏置
    b1 = np.random.rand(numHide) 
    b2 = np.random.rand(numOut) 
    w1 = np.random.rand(numIn,numHide) 
    w2 = np.random.rand(numHide,numOut)
    
    # 通过字典存储参数
    parameters = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}
    return parameters

# 定义 sigmoid函数
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

# 前向传播
def forward_propagation(x, parameters):
    # w1和b1为输入层与隐藏层之间的权重和偏置，w2和b2为隐藏层与输出层之间的权重和偏置
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    # 使用sigmoid函数作为激活函数，a1和z1为输入层与隐藏层之间的输入和输出，w2和b2为隐藏层与输出层之间的输入和输出
    a1 = np.dot(x,w1)-b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,w2)-b2 
    z2 = sigmoid(a2)

    cache = {'a1': a1, 'z1': z1, 'a2': a2, 'z2': z2}
    return z2,cache

# 计算损失
def loss(z2, y, m):
    # 采用交叉熵函数作为损失函数
    cost = - np.sum(np.multiply(np.log(z2), y) + np.multiply((1 - y), np.log(1 - z2))) / m
    return cost

# 反向传播
def backward_propagation(parameters, cache, x, y, n):
    w2 = parameters['w2']
    z1 = cache['z1']
    z2 = cache['z2']

    # 根据链式法则推导求导公式，计算e_w1、e_b1、e_w2、e_b2
    g = z2*(1-z2)*(y-z2) 
    e_w2 = np.dot(z1.T,g)
    e_b2 = -np.mean(g, axis=0)

    e = z1*(1-z1)*np.dot(g,w2.T)
    e_w1 = np.dot(x.T, e) 
    e_b1= -np.mean(e, axis=0)
    
    grads = {'e_w1': e_w1, 'e_b1': e_b1, 'e_w2': e_w2, 'e_b2': e_b2}
    return grads

# 更新参数
def update_parameters(parameters, grads, learning_rate):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    e_w1 = grads['e_w1']
    e_b1 = grads['e_b1']
    e_w2 = grads['e_w2']
    e_b2 = grads['e_b2']

    w1 = w1 + e_w1 * learning_rate
    b1 = b1 + e_b1 * learning_rate
    w2 = w2 + e_w2 * learning_rate
    b2 = b2 + e_b2 * learning_rate

    parameters = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2}
    return parameters

# 测试函数
def test(parameters, x_test, y_test, m):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']

    # 使用训练后得到的参数进行前向传播得到预测值
    a1 = np.dot(x_test,w1)-b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,w2)-b2 
    z2 = sigmoid(a2)
    
    # 对预测值分类
    output = np.zeros((m, 3), dtype=int)

    for i in range(m):
        for j in range(3):
            if z2[i][j] > 0.5:
                output[i][j] = 1
            else:
                output[i][j] = 0
                
    yuce=np.zeros(m,dtype=int)  
    
    for i in range(m):
        if output[i][0]==1 and output[i][1]==0 and output[i][2]==0:
            yuce[i]=1
        elif output[i][0]==0 and output[i][1]==1 and output[i][2]==0:
            yuce[i]=2
        elif output[i][0]==0 and output[i][1]==0 and output[i][2]==1:
            yuce[i]=3

    # 计算正确率
    right_num = 0
    for j in range(m):
        if yuce[j] == y_test[j] :
            right_num = right_num + 1
    right_rate = right_num / m 

    return right_rate

# 顶层模块，调用各函数组成BP神经网络模型
def neural_model(x, y, x_test, y_test, numIn, numHide, numOut, numIterations): 
    print('输入层结点数：%i  隐藏层结点数：%i  输出层结点数：%i  训练次数：%i' %(numIn,numHide,numOut,numIterations))
    n = x.shape[0]
    m = x_test.shape[0]
    # 初始化参数
    parameters = initialize_parameters(numIn, numHide, numOut)
    for i in range(numIterations):
        # 前向传播
        z2, cache = forward_propagation(x, parameters)
        # 计算损失
        cost = loss(z2, y, n)
        # 反向传播
        grads = backward_propagation(parameters, cache, x, y, n)
        # 更新参数
        parameters = update_parameters(parameters, grads, 0.001)
        # 进行测试
        
        # 打印输出损失和正确率
        if (i + 1) % 500 == 0:         
            right_rate = test(parameters, x_test, y_test, m)
            print('训练%i次，损失为：%f  准确率：%.2f' % (i+1,cost,right_rate))
            
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']   
    print('训练后输入层与隐藏层之间的权值为：')
    print(w1)
    print('训练后输入层与隐藏层之间的偏置为：')
    print(b1.T)
    print('训练后隐藏层与输出层之间的权值为：')
    print(w2)
    print('训练后隐藏层与输出层之间的偏置为：')
    print(b2.T)


if __name__ == "__main__":
    # 读取数据
    x = []
    y = []
    result = []
    with open('鸢尾花数据集.txt', 'r') as f:
        data = f.readlines()
    # 打乱数据顺序
    random.shuffle(data)
    # 读取参数x，结果y，值result
    for line in data:
        values = line.strip().split(',')
        x.append([float(v) for v in values[:-1]])
        if values[4] == 'setosa':  # setosa赋值1
            y.append([1,0,0])
            result.append(1)
        elif values[4] == 'versicolor':  # versicolor赋值2
            y.append([0,1,0])
            result.append(2)
        elif values[4] == 'virginica':  # virginica赋值3
            y.append([0,0,1])
            result.append(3)

    x = np.array(x)
    y = np.array(y)
    result = np.array(result)  
    # 划分训练集和测试集
    x_train = x[:120]
    y_train = y[:120]
    x_test = x[120:]
    y_test = result[120:]

    # 使用BP神经网络模型进行训练，输入层4个结点，隐藏层1个结点，输出层1个结点，训练200次
    parameters = neural_model(x, y, x_test, y_test, 4, 10, 3, 3000)
    

输入层结点数：4  隐藏层结点数：10  输出层结点数：3  训练次数：3000
训练500次，损失为：1.409344  准确率：0.57
训练1000次，损失为：1.053660  准确率：0.67
训练1500次，损失为：0.851646  准确率：0.83
训练2000次，损失为：0.663516  准确率：0.97
训练2500次，损失为：0.528269  准确率：0.97
训练3000次，损失为：0.442659  准确率：0.97
训练后输入层与隐藏层之间的权值为：
[[ 0.65843167 -1.62704722  0.87164282 -0.45133868  0.3900633  -1.18940435
   1.00427913  0.80202447 -0.22162126  0.70354138]
 [ 0.24457636 -1.65966204  0.24634784 -0.9173418   1.14898661 -0.47759174
   0.28912229  0.20637151 -0.83553253  0.56058568]
 [ 0.20134475  2.43309329  1.00064714  1.98516169 -1.8324379   1.22925721
   0.78151734  0.01114742  1.3515743   0.58909234]
 [ 0.1709534   2.3608441   0.59836138  0.92092043 -0.44340953  1.86384225
   0.39300065  0.28317122  1.2459687   0.92072782]]
训练后输入层与隐藏层之间的偏置为：
[0.38785268 0.82871958 0.53820543 0.47369905 0.52625045 0.27003703
 0.21154925 0.94147836 0.19381407 0.85657204]
训练后隐藏层与输出层之间的权值为：
[[-0.18122761 -0.54510793 -0.22850712]
 [-1.75854509 -2.85678756  3.65217066]
 [ 0.49507284  0.20185519 -0